In [253]:
from DealMatch.trainer_unsupervised import Trainer
from DealMatch.data_unsupervised import get_targets_data, get_investors_data, get_matching_keys, clean_targets, clean_investors
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import RobustScaler
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
from sklearn.base import TransformerMixin
from sklearn.neighbors import NearestNeighbors, KNeighborsClassifier


In [24]:
df_targets_clean = pd.read_csv('../targets.csv', index_col=0).drop(columns='index')
df_investors_clean = pd.read_csv('../investors.csv', index_col=0)
test = pd.read_excel('../DealMatch/targets_clean_test.xlsx')

In [25]:
df_targets_clean.head()

,deal_id,deal_name,deal_type_name,target_company_id,target_name,target_description,target_revenue,target_ebitda,target_ebit,country_name,region_name,sector_name,strs
0,173,1301 Sun,OTHER,871,Sun [Target],•\t350MWp of solar photovoltaic project assets...,NaN,NaN,NaN,NaN,NaN,Energy,energy solar power energie photovoltaik pv...
4,129,1220 Supple,OTHER,874,Supple [Target],Solaranlage in Kreta,57.70,NaN,NaN,NaN,NaN,Energy,energy renewable energy other solar power ...
16,407,1677 Heat,MAJORITY,806,SPH Sustainable Process Heat GmbH,PROJECT HEAT hat eine neue Wärmepumpentechnolo...,0.43,-0.78,-0.78,NaN,NaN,Industrial products and services,industrial products and services pumps and co...
22,1013,845 Apollo,OTHER,812,SUMMIQ AG,Fundraising für Renewable Holding,0.00,0.00,0.00,Germany,Bavaria,Financial Services,financial services other diversified financia...
26,752,390 Saragossa,OTHER,507,KSW Bioenergie GmbH,Errichtung eines CO2-neutralen Bio-Energie Kra...,0.00,0.00,0.00,Germany,North Rhine-Westphalia,Energy,bioenergy energy bioenergie biomasse biotr...


In [26]:
df_investors_clean.head()

,name,name_de
0,10X group,startup app daten internet agnostisch
1,123 Investment Managers,tourismus altenpflege gastronomie labor senior...
2,137 Ventures,agnostisch
3,138 Pyramids,agnostisch
4,17Capital LLP,agnostisch


In [42]:
X = df_targets_clean
Y = df_investors_clean
model = Trainer(X,Y)

In [43]:
model.set_pipeline_targets()

In [44]:
model.run_targets()

In [52]:
num_transformer = Pipeline([('imputer', SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=0)),
                            ('scaler', RobustScaler())])

In [54]:
preprocessor = ColumnTransformer(transformers=[
            ('num_tr', num_transformer, ['target_ebit','target_ebitda','target_revenue']),
            ('tfidf',TfidfVectorizer(), 'strs')
        ], remainder='drop')

In [55]:
pipeline_targets = Pipeline([('preproc', preprocessor)])

In [58]:
pipeline_targets.fit_transform(X)

<1111x4632 sparse matrix of type '<class 'numpy.float64'>'
	with 17306 stored elements in Compressed Sparse Row format>

In [62]:
preproc_x = preprocessor.fit_transform(X)

In [130]:
num_features = ['target_ebit','target_ebitda','target_revenue']
num_transformer = Pipeline([('imputer', SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=0)),
                            ('scaler', RobustScaler())])

In [194]:
class DenseTransformer(TransformerMixin):

    def fit(self, X, y=None, **fit_params):
        return self

    def transform(self, X, y=None, **fit_params):
        return X.toarray()

In [195]:
tfidf_features = 'strs'
tfidf_transformer = Pipeline([('tfidf', TfidfVectorizer()), ('dense', DenseTransformer())])

In [196]:
preproc = ColumnTransformer(transformers=[
            ('num_tr', num_transformer, num_features),
            ('tfidf',tfidf_transformer, tfidf_features)
        ], remainder='drop')

In [197]:
preproc.fit_transform(X).shape

(1111, 4632)

In [167]:
tf = tfidf_transformer.fit_transform(X['strs']).toarray()

In [168]:
num = num_transformer.fit_transform(X[['target_ebit','target_ebitda','target_revenue']])

In [173]:
merge = np.concatenate((tf,num), axis=1)

In [269]:
full = Pipeline([('preproc', preproc),
                                          ('pca',
                                           PCA(0.95))])

In [272]:
preproc_fitted = full.fit(X)
preproc_transformed = preproc_fitted.transform(X)
fitted_nn = NearestNeighbors(n_neighbors=10).fit(preproc_transformed)

In [273]:
test_transformed = preproc_fitted.transform(test)
fitted_nn.kneighbors(test_transformed)

(array([[0.11123943, 0.27253504, 0.29032934, 0.45281042, 0.48279175,
         0.48902642, 0.57433349, 0.60868525, 0.63310101, 0.65931131]]),
 array([[451, 648, 729,  72, 886, 877, 136, 893,  66, 319]]))

In [271]:
transformed_test = fitted.transform(test)

In [ ]:
nn = NearestNeighbors(n_neighbors=10)

In [237]:
transformed_test = fitted['preproc'].transform(test)

In [241]:
fit_transform = fitted['pca'].transform(transformed_test)

In [242]:
fitted['NN'].kneighbors(fit_transform)

(array([[0.11123943, 0.27253504, 0.29032934, 0.45281042, 0.48279175,
         0.48902642, 0.57433349, 0.60868525, 0.63310101, 0.65931131]]),
 array([[451, 648, 729,  72, 886, 877, 136, 893,  66, 319]]))

In [268]:
fitted['NN'].kneighbors(test,10)

/Users/michieldekoninck/.pyenv/versions/3.8.12/envs/dealmatch/lib/python3.8/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but NearestNeighbors was fitted without feature names
  warnings.warn(


ValueError: could not convert string to float: '1303 Sonne'